In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
import sys
import cv2
sys.path.insert(0, '../codes')
import video_utils
from skimage.measure import compare_ssim as ssim

/home/wesley.passos/miniconda3/envs/mosquitoes/lib/python3.6/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [2]:
# the notebook will reload external python modules;
%load_ext autoreload
%autoreload 2

In [3]:
#make matplotlib figures appear inline in the notebook
#rather than in a new window.
%matplotlib inline
#sns.set()
# set default size of plots
plt.rcParams['figure.figsize'] = (12.0, 10.0) 
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
plt.rcParams['xtick.labelsize'] = 20
plt.rcParams['ytick.labelsize'] = 20


#print OpenCV version
print ("OpenCV Version : %s " % cv2.__version__)

OpenCV Version : 3.3.1 


In [4]:
def mse(x, y):
    return np.linalg.norm(x - y)/(x.shape[0]*x.shape[1])

## Creating video object

In [7]:
videos_path = '../data/DJI4_cam/2018-09-05/seq001/mission_001'
video_name = 'original_001.MOV'
video = os.path.join(videos_path, video_name)
vid = video_utils.videoObj(videopath=video)

In [9]:
vid.videoInfo.printAllInformation()


*************************************
************* video info ************
 
File path: ../data/DJI4_cam/2018-09-05/seq001/mission_001/original_001.MOV
File name: original_001.MOV
File extension: mov,mp4,m4a,3gp,3g2,mj2 (QuickTime / MOV)
Created on: 2018-09-05T11:14:19.000000Z
File size: 1962746669
Codec: h264 (H.264 / AVC / MPEG-4 AVC / MPEG-4 part 10)
Width: 3840
Height: 2160
Width x Height: [3840, 2160]
Sample aspect ratio: 1:1
Display aspect ratio: 16:9
Pixel format: yuv420p
Frame rate: 48000/1001
Duration ts: 7537530
Duration: 157.031875
Bit rate: 99963919
Number of frames: 7530

*************************************


In [ ]:
print('rms: {}'.format(ret))
print('camera matrix: \n', mtx)
print('distortion coefficients: ', dist)

## Testing calibration

In [ ]:
#load video
video_test = os.path.join(videos_path,video_name)
video_capture = cv2.VideoCapture(video_test)

#extract first frame
success,img = video_capture.read()
h, w = img.shape[:2]

#displaying image
plt.imshow(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.title('original image')
plt.show()
print('image size: ', img.shape)

* optimizing camera params:

In [ ]:
#optimize camera matrix
newcameramtx, roi=cv2.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))

print('\nCamera matrix: \n', mtx)
print('\nNew camera matrix: \n', newcameramtx)
print('\nRoI: \n', roi)

* undistorting frame:

In [ ]:
#undistort
dst = cv2.undistort(img, mtx, dist, None, newcameramtx)

plt.imshow(cv2.cvtColor(dst,cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.title('undistorted image')
plt.show()
# print('image size: ', dst.shape)

* computing SSIM and MSE between images...

In [ ]:
MSE = mse(img, dst)
SSIM = ssim(img, dst, multichannel=True)

print('MSE: ', MSE)
print('SSIM: ', SSIM)

* blending original and undistorted images...

In [ ]:
alpha = 0.5
beta = (1.0 - alpha)
blend = np.uint8(alpha*(img)+beta*(dst))

plt.imshow(cv2.cvtColor(blend,cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.title('Blended images')
plt.show()